In [6]:
import requests
import json

page = requests.get("https://api.datamuse.com/words?rel_rhy=funny")
print(type(page))
print(page.text[:150]) # print the first 150 characters
print(page.url) # print the url that was fetched
print("------")
x = page.json() # turn page.text into a python object
print(type(x))
print("---first item in the list---")
print(x[0])
print("---the whole list, pretty printed---\n\n")
print(type(json.dumps(x, indent=2)))
print(json.dumps(x, indent=2)) # pretty print the results

<class 'requests.models.Response'>
[{"word":"money","score":4415,"numSyllables":2},{"word":"honey","score":1206,"numSyllables":2},{"word":"sunny","score":718,"numSyllables":2},{"word":"
https://api.datamuse.com/words?rel_rhy=funny
------
<class 'list'>
---first item in the list---
{'word': 'money', 'score': 4415, 'numSyllables': 2}
---the whole list, pretty printed---


<class 'str'>
[
  {
    "word": "money",
    "score": 4415,
    "numSyllables": 2
  },
  {
    "word": "honey",
    "score": 1206,
    "numSyllables": 2
  },
  {
    "word": "sunny",
    "score": 718,
    "numSyllables": 2
  },
  {
    "word": "bunny",
    "score": 702,
    "numSyllables": 2
  },
  {
    "word": "blini",
    "score": 613,
    "numSyllables": 2
  },
  {
    "word": "gunny",
    "score": 449,
    "numSyllables": 2
  },
  {
    "word": "tunny",
    "score": 301,
    "numSyllables": 2
  },
  {
    "word": "sonny",
    "score": 286,
    "numSyllables": 2
  },
  {
    "word": "dunny",
    "score": 245,
    "n

Here’s an executable sample, using the optional params parameter of requests.get. It gets the same data from the datamus api that we saw previously. Here, however, the full url is built inside the call to requests.get; we can see what url was built by printing it out, on line 5.

In [7]:
import requests

# page = requests.get("https://api.datamuse.com/words?rel_rhy=funny")
kval_pairs = {'rel_rhy': 'funny'}
page = requests.get("https://api.datamuse.com/words", params=kval_pairs)
print(page.text[:150]) # print the first 150 characters
print(page.url) # print the url that was fetched

[{"word":"money","score":4415,"numSyllables":2},{"word":"honey","score":1206,"numSyllables":2},{"word":"sunny","score":718,"numSyllables":2},{"word":"
https://api.datamuse.com/words?rel_rhy=funny


In [15]:
#d = {'q': '"violins and guitars"', 'tbm': 'isch'}
#results = requests.get("https://google.com/search", params=d)
print(results.url)
print(type(results))


https://www.google.com/search?q=%22violins+and+guitars%22&tbm=isch
<class 'requests.models.Response'>


In [29]:
# import statements for necessary Python modules
import requests

def get_rhymes(word):
    baseurl = "https://api.datamuse.com/words"
    params_diction = {} # Set up an empty dictionary for query parameters
    params_diction["rel_rhy"] = word
    params_diction["max"] = "3" # get at most 3 results
    resp = requests.get(baseurl, params=params_diction)
    # return the top three words
    word_ds = resp.json()
    print(word_ds)
    return [d['word'] for d in word_ds]
    return resp.json() # Return a python object (a list of dictionaries in this case)

print(get_rhymes("funny"))
print(get_rhymes("honey"))
print(get_rhymes("herry")) 

[{'word': 'money', 'score': 4415, 'numSyllables': 2}, {'word': 'honey', 'score': 1206, 'numSyllables': 2}, {'word': 'sunny', 'score': 718, 'numSyllables': 2}]
['money', 'honey', 'sunny']
[{'word': 'money', 'score': 4415, 'numSyllables': 2}, {'word': 'funny', 'score': 1265, 'numSyllables': 2}, {'word': 'sunny', 'score': 718, 'numSyllables': 2}]
['money', 'funny', 'sunny']
[{'word': 'dictionary', 'score': 9333, 'numSyllables': 4}, {'word': 'cherry', 'score': 2903, 'numSyllables': 2}, {'word': 'arbitrary', 'score': 2854, 'numSyllables': 4}]
['dictionary', 'cherry', 'arbitrary']


In [7]:
import requests
import json

PERMANENT_CACHE_FNAME = "permanent_cache.txt"
TEMP_CACHE_FNAME = "this_page_cache.txt"

def _write_to_file(cache, fname):
    with open(fname, 'w') as outfile:
        outfile.write(json.dumps(cache, indent=2))

def _read_from_file(fname):
    try:
        with open(fname, 'r') as infile:
            res = infile.read()
            return json.loads(res)
    except:
        return {}

def add_to_cache(cache_file, cache_key, cache_value):
    temp_cache = _read_from_file(cache_file)
    temp_cache[cache_key] = cache_value
    _write_to_file(temp_cache, cache_file)

def clear_cache(cache_file=TEMP_CACHE_FNAME):
    _write_to_file({}, cache_file)

def make_cache_key(baseurl, params_d, private_keys=["api_key"]):
    """Makes a long string representing the query.
    Alphabetize the keys from the params dictionary so we get the same order each time.
    Omit keys with private info."""
    alphabetized_keys = sorted(params_d.keys())
    res = []
    for k in alphabetized_keys:
        if k not in private_keys:
            res.append("{}-{}".format(k, params_d[k]))
    return baseurl + "_".join(res)

def get(baseurl, params={}, private_keys_to_ignore=["api_key"], permanent_cache_file=PERMANENT_CACHE_FNAME, temp_cache_file=TEMP_CACHE_FNAME):
    full_url = requests.requestURL(baseurl, params)
    cache_key = make_cache_key(baseurl, params, private_keys_to_ignore)
    # Load the permanent and page-specific caches from files
    permanent_cache = _read_from_file(permanent_cache_file)
    temp_cache = _read_from_file(temp_cache_file)
    if cache_key in temp_cache:
        print("found in temp_cache")
        # make a Response object containing text from the change, and the full_url that would have been fetched
        return requests.Response(temp_cache[cache_key], full_url)
    elif cache_key in permanent_cache:
        print("found in permanent_cache")
        # make a Response object containing text from the change, and the full_url that would have been fetched
        return requests.Response(permanent_cache[cache_key], full_url)
    else:
        print("new; adding to cache")
        # actually request it
        resp = requests.get(baseurl, params)
        # save it
        add_to_cache(temp_cache_file, cache_key, resp.text)
        return resp

In [8]:
import requests_with_caching
# it's not found in the permanent cache
res = requests_with_caching.get("https://api.datamuse.com/words?rel_rhy=happy", permanent_cache_file="datamuse_cache.txt")
print(res.text[:100])
# this time it will be found in the temporary cache
res = requests_with_caching.get("https://api.datamuse.com/words?rel_rhy=happy", permanent_cache_file="datamuse_cache.txt")
# This one is in the permanent cache.
res = requests_with_caching.get("https://api.datamuse.com/words?rel_rhy=funny", permanent_cache_file="datamuse_cache.txt")


ModuleNotFoundError: No module named 'requests_with_caching'

In [13]:
import requests
import json

parameters = {"term": "Ann Arbor", "entity": "podcast"}
iTunes_response = requests.get("https://itunes.apple.com/search", params = parameters)

print(iTunes_response.url)
py_data = json.loads(iTunes_response.text)
for r in py_data['results']:
    print(r['trackName'])

NameError: name 'iTunes_response' is not defined

In [ ]:
# import statements
import requests_with_caching
import json
# import webbrowser

# apply for a flickr authentication key at http://www.flickr.com/services/apps/create/apply/?
# paste the key (not the secret) as the value of the variable flickr_key
flickr_key = 'yourkeyhere'

def get_flickr_data(tags_string):
    baseurl = "https://api.flickr.com/services/rest/"
    params_diction = {}
    params_diction["api_key"] = flickr_key # from the above global variable
    params_diction["tags"] = tags_string # must be a comma separated string to work correctly
    params_diction["tag_mode"] = "all"
    params_diction["method"] = "flickr.photos.search"
    params_diction["per_page"] = 5
    params_diction["media"] = "photos"
    params_diction["format"] = "json"
    params_diction["nojsoncallback"] = 1
    flickr_resp = requests_with_caching.get(baseurl, params = params_diction, permanent_cache_file="flickr_cache.txt")
    # Useful for debugging: print the url! Uncomment the below line to do so.
    print(flickr_resp.url) # Paste the result into the browser to check it out...
    return flickr_resp.json()

result_river_mts = get_flickr_data("river,mountains")

# Some code to open up a few photos that are tagged with the mountains and river tags...

photos = result_river_mts['photos']['photo']
for photo in photos:
    owner = photo['owner']
    photo_id = photo['id']
    url = 'https://www.flickr.com/photos/{}/{}'.format(owner, photo_id)
    print(url)
    # webbrowser.open(url)

In [ ]:
webbrowser.open(https://www.flickr.com/photos/145056248@N07/44857423045)

In [55]:

# some invocations that we use in the automated tests; uncomment these if you are getting errors and want better error messages
import requests
import json
def get_movies_from_tastedive(content):
    baseurl="https://tastedive.com/api/similar"
    parameters={}
    parameters["q"]=content
    parameters["type"]="movies"
    parameters["limit"]=5
    parameters['k']="378766-rharod4g-EVKG8KEA"
    movie_resp=requests.get(baseurl,params=parameters)
    print(movie_resp.url)
    print(movie_resp.status_code)
    return movie_resp.json()


def extract_movie_titles(movie_dict):
    return [d['Name'] for d in movie_dict['Similar']['Results']]

#print(extract_movie_titles(get_movies_from_tastedive("Tony Bennett")))
#print(extract_movie_titles(get_movies_from_tastedive("Bridesmaids")))



def get_related_titles(lst):
    L1=extract_movie_titles(get_movies_from_tastedive(lst[0]))
    L2=extract_movie_titles(get_movies_from_tastedive(lst[1]))
    print(L1)
    print(L2)
    [L1.append(d) for d in L2 if d not in L1]
    return L1

print(get_related_titles(["Black Panther", "Captain Marvel"]))

https://tastedive.com/api/similar?q=Black+Panther&type=movies&limit=5&k=378766-rharod4g-EVKG8KEA
200
https://tastedive.com/api/similar?q=Captain+Marvel&type=movies&limit=5&k=378766-rharod4g-EVKG8KEA
200
[]
['Spider-Man: Far From Home', 'Ant-Man And The Wasp', 'Aquaman', 'Shazam!', 'Toy Story 4']
['Spider-Man: Far From Home', 'Ant-Man And The Wasp', 'Aquaman', 'Shazam!', 'Toy Story 4']


In [ ]:

# some invocations that we use in the automated tests; uncomment these if you are getting errors and want better error messages

# some invocations that we use in the automated tests; uncomment these if you are getting errors and want better error messages
import requests_with_caching
import json
def get_movies_from_tastedive(content):
    baseurl="https://tastedive.com/api/similar"
    parameters={}
    parameters["q"]=content
    parameters["type"]="movies"
    parameters["limit"]=5
    movie_resp=requests_with_caching.get(baseurl,params=parameters)
    return movie_resp.json()

def extract_movie_titles(movie_dict):
    return [d['Name'] for d in movie_dict['Similar']['Results']]


extract_movie_titles(get_movies_from_tastedive("Tony Bennett"))
extract_movie_titles(get_movies_from_tastedive("Black Panther"))




In [52]:
l1=["ram","shyam","sita"]
l2=["hey","sita","hello"]
L= [l1.append(d) for d in l2 if d not in l1]
l1

['ram', 'shyam', 'sita', 'hey', 'hello']

In [61]:
lst=["Black Panther", "Captain Marvel"]
len(lst)

2

In [ ]:

# some invocations that we use in the automated tests; uncomment these if you are getting errors and want better error messages

# some invocations that we use in the automated tests; uncomment these if you are getting errors and want better error messages
import requests_with_caching
import json
def get_movies_from_tastedive(content):
    baseurl="https://tastedive.com/api/similar"
    parameters={}
    parameters["q"]=content
    parameters["type"]="movies"
    parameters["limit"]=5
    movie_resp=requests_with_caching.get(baseurl,params=parameters)
    return movie_resp.json()

def extract_movie_titles(movie_dict):
    return [d['Name'] for d in movie_dict['Similar']['Results']]

def get_related_titles(L):
    if len(L)>0:
        L1=[]
        for d in L:
            lst=extract_movie_titles(get_movies_from_tastedive(d))
            for m in lst:
                if m not in L1:
                    L1.append(m)
        return list(L1)
    else:
        return []


get_related_titles(["Black Panther", "Captain Marvel"])
#get_related_titles([])


In [ ]:

# some invocations that we use in the automated tests; uncomment these if you are getting errors and want better error messages
import requests_with_caching
import json
def get_movie_data(content):
    baseurl="http://www.omdbapi.com/"
    parameters={}
    parameters["t"]=content
    parameters["r"]='json'
    movie_resp=requests_with_caching.get(baseurl,params=parameters)
    return movie_resp.json()

get_movie_data("Venom")
get_movie_data("Baby Mama")
# get_movie_data("Venom")
# get_movie_data("Baby Mama")

In [62]:
L={'Response': 'True', 'Title': 'Deadpool 2', 'Year': '2018', 'Rated': 'R', 'Released': '18 May 2018', 'Runtime': '119 min', 'Genre': 'Action, Adventure, Comedy, Sci-Fi', 'Director': 'David Leitch', 'Writer': 'Rhett Reese, Paul Wernick, Ryan Reynolds', 'Actors': 'Ryan Reynolds, Josh Brolin, Morena Baccarin, Julian Dennison', 'Plot': 'Foul-mouthed mutant mercenary Wade Wilson (AKA. Deadpool), brings together a team of fellow mutant rogues to protect a young boy with supernatural abilities from the brutal, time-traveling cyborg, Cable.', 'Language': 'English, Cantonese, Spanish, Russian', 'Country': 'USA', 'Awards': 'N/A', 'Poster': 'https://m.media-amazon.com/images/M/MV5BNjk1Njk3YjctMmMyYS00Y2I4LThhMzktN2U0MTMyZTFlYWQ5XkEyXkFqcGdeQXVyODM2ODEzMDA@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.8/10'}, {'Source': 'Rotten Tomatoes', 'Value': '83%'}, {'Source': 'Metacritic', 'Value': '66/100'}], 'Metascore': '66', 'imdbRating': '7.8', 'imdbVotes': '311,435', 'imdbID': 'tt5463162', 'Type': 'movie', 'DVD': '21 Aug 2018', 'BoxOffice': 'N/A', 'Production': '20th Century Fox', 'Website': 'https://www.foxmovies.com/movies/deadpool-2'}

In [63]:
print(json.dumps(L,indent=2))

{
  "Response": "True",
  "Title": "Deadpool 2",
  "Year": "2018",
  "Rated": "R",
  "Released": "18 May 2018",
  "Runtime": "119 min",
  "Genre": "Action, Adventure, Comedy, Sci-Fi",
  "Director": "David Leitch",
  "Writer": "Rhett Reese, Paul Wernick, Ryan Reynolds",
  "Actors": "Ryan Reynolds, Josh Brolin, Morena Baccarin, Julian Dennison",
  "Plot": "Foul-mouthed mutant mercenary Wade Wilson (AKA. Deadpool), brings together a team of fellow mutant rogues to protect a young boy with supernatural abilities from the brutal, time-traveling cyborg, Cable.",
  "Language": "English, Cantonese, Spanish, Russian",
  "Country": "USA",
  "Awards": "N/A",
  "Poster": "https://m.media-amazon.com/images/M/MV5BNjk1Njk3YjctMmMyYS00Y2I4LThhMzktN2U0MTMyZTFlYWQ5XkEyXkFqcGdeQXVyODM2ODEzMDA@._V1_SX300.jpg",
  "Ratings": [
    {
      "Source": "Internet Movie Database",
      "Value": "7.8/10"
    },
    {
      "Source": "Rotten Tomatoes",
      "Value": "83%"
    },
    {
      "Source": "Metacritic"

In [67]:
print(L["Ratings"][1]['Value'][:-1])

83


In [79]:
def rat(srt):
    for d in srt["Ratings"]:
        if 'Rotten Tomatoes' in d['Source']:
            return d['Value'][:-1]
    else:
        return 0

print(rat(L))

83


In [ ]:

# some invocations that we use in the automated tests; uncomment these if you are getting errors and want better error messages
import requests_with_caching
import json
def get_movie_data(content):
    baseurl="http://www.omdbapi.com/"
    parameters={}
    parameters["t"]=content
    parameters["r"]='json'
    movie_resp=requests_with_caching.get(baseurl,params=parameters)
    return movie_resp.json()

def get_movie_rating(srt):
    for d in srt["Ratings"]:
        if 'Rotten Tomatoes' in d['Source']:
            return int(d['Value'][:-1])
    else:
        return 0
    
    
get_movie_rating(get_movie_data("Deadpool 2"))

In [85]:
l=["ram","shyam","sita"]
r=[1,2,3]
L=zip(l,r)

In [86]:

d={}
for x,y in L:
    d[x]=y

In [87]:
d

{'ram': 1, 'shyam': 2, 'sita': 3}

In [ ]:

# some invocations that we use in the automated tests; uncomment these if you are getting errors and want better error messages
import requests_with_caching
import json
import sys
sys.setExecutionLimit(100000) 
def get_movies_from_tastedive(content):
    baseurl="https://tastedive.com/api/similar"
    parameters={}
    parameters["q"]=content
    parameters["type"]="movies"
    parameters["limit"]=5
    movie_resp=requests_with_caching.get(baseurl,params=parameters)
    return movie_resp.json()

def extract_movie_titles(movie_dict):
    return [d['Name'] for d in movie_dict['Similar']['Results']]

def get_related_titles(L):
    if len(L)>0:
        L1=[]
        for d in L:
            lst=extract_movie_titles(get_movies_from_tastedive(d))
            for m in lst:
                if m not in L1:
                    L1.append(m)
        return list(L1)
    else:
        return []
    

def get_movie_data(content):
    baseurl="http://www.omdbapi.com/"
    parameters={}
    parameters["t"]=content
    parameters["r"]='json'
    movie_resp=requests_with_caching.get(baseurl,params=parameters)
    return movie_resp.json()

def get_movie_rating(srt):
    for d in srt["Ratings"]:
        if 'Rotten Tomatoes' in d['Source']:
            return int(d['Value'][:-1])
    else:
        return 0
    
def get_sorted_recommendations(words):
    list_of_movies=get_related_titles(words)
    rate=[]
    for d in list_of_movies:
        rate.append(get_movie_rating(get_movie_data(d)))
    new=zip(list_of_movies,rate)
    d={}
    for name,r in new:
        d[name]=r
    return sorted(d,key=lambda x:(-d[x],d))
    


get_sorted_recommendations(["Bridesmaids", "Sherlock Holmes"])